## 1. Càrrega de Llibreries

In [1]:
# Llibreries de dades
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Llibreries de Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Serialització
import pickle
import os

# Configuració de visualització
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 2. Càrrega i Exploració de les Dades (EDA)

### 2.1 Carregar el Dataset

In [2]:
# Carregar dataset des de Seaborn
df = sns.load_dataset("penguins")

# Mostrar primeres files
print(f"Dataset carregat amb {len(df)} files i {len(df.columns)} columnes")
df.head(10)

Dataset carregat amb 344 files i 7 columnes


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,Female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,Male
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN


In [3]:
# Informació del dataset
print("="*50)
print("INFORMACIÓ DEL DATASET")
print("="*50)
df.info()

INFORMACIÓ DEL DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
# Estadístiques descriptives
print("="*50)
print("ESTADÍSTIQUES DESCRIPTIVES")
print("="*50)
df.describe()

ESTADÍSTIQUES DESCRIPTIVES


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [5]:
# Distribució de la variable objectiu (species)
print("="*50)
print("DISTRIBUCIÓ D'ESPÈCIES")
print("="*50)
print(df['species'].value_counts())
print("\nProporció:")
print(df['species'].value_counts(normalize=True).round(3))

DISTRIBUCIÓ D'ESPÈCIES
species
Adelie       152
Gentoo       124
Chinstrap     68
Name: count, dtype: int64

Proporció:
species
Adelie       0.442
Gentoo       0.360
Chinstrap    0.198
Name: proportion, dtype: float64


In [6]:
# Valors nuls
print("="*50)
print("VALORS NULS PER COLUMNA")
print("="*50)
null_counts = df.isnull().sum()
print(null_counts)
print(f"\nTotal de files amb algun valor nul: {df.isnull().any(axis=1).sum()}")
print(f"Total de files: {len(df)}")

VALORS NULS PER COLUMNA
species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

Total de files amb algun valor nul: 11
Total de files: 344


In [7]:
# Valors únics de columnes categòriques (important per al client!)
print("="*50)
print("VALORS ÚNICS DE VARIABLES CATEGÒRIQUES")
print("="*50)
print(f"species: {df['species'].unique().tolist()}")
print(f"island: {df['island'].unique().tolist()}")
print(f"sex: {df['sex'].dropna().unique().tolist()}")

VALORS ÚNICS DE VARIABLES CATEGÒRIQUES
species: ['Adelie', 'Chinstrap', 'Gentoo']
island: ['Torgersen', 'Biscoe', 'Dream']
sex: ['Male', 'Female']


## 3. Preprocessament de les Dades

### 3.1 Eliminar files amb valors nuls

In [8]:
# Eliminar files amb NA
df_clean = df.dropna()

print(f"Files originals: {len(df)}")
print(f"Files després d'eliminar NA: {len(df_clean)}")
print(f"Files eliminades: {len(df) - len(df_clean)}")

# Verificar que no queden valors nuls
print(f"\nValors nuls restants: {df_clean.isnull().sum().sum()}")

Files originals: 344
Files després d'eliminar NA: 333
Files eliminades: 11

Valors nuls restants: 0


### 3.2 Separar features i target

In [9]:
# Separar features (X) i target (y)
X = df_clean.drop('species', axis=1)
y = df_clean['species']

# Definir variables categòriques i numèriques
cat_cols = ['island', 'sex']
num_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

print("Variables predictores (features):")
print(f"  Categòriques: {cat_cols}")
print(f"  Numèriques: {num_cols}")
print(f"\nVariable objectiu: species")
print(f"\nForma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Variables predictores (features):
  Categòriques: ['island', 'sex']
  Numèriques: ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

Variable objectiu: species

Forma de X: (333, 6)
Forma de y: (333,)


### 3.3 Divisió Train/Test (80/20)

**IMPORTANT:** La divisió es fa ABANS del preprocessament per evitar data leakage.

In [10]:
# Divisió 80/20 ABANS de preprocessar (evitar data leakage!)
# stratify=y manté les proporcions d'espècies en train i test
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Divisió Train/Test:")
print(f"  Train: {X_train_raw.shape[0]} mostres ({X_train_raw.shape[0]/len(X)*100:.1f}%)")
print(f"  Test: {X_test_raw.shape[0]} mostres ({X_test_raw.shape[0]/len(X)*100:.1f}%)")

print("\nDistribució d'espècies en Train:")
print(y_train.value_counts())

print("\nDistribució d'espècies en Test:")
print(y_test.value_counts())

Divisió Train/Test:
  Train: 266 mostres (79.9%)
  Test: 67 mostres (20.1%)

Distribució d'espècies en Train:
species
Adelie       117
Gentoo        95
Chinstrap     54
Name: count, dtype: int64

Distribució d'espècies en Test:
species
Adelie       29
Gentoo       24
Chinstrap    14
Name: count, dtype: int64


### 3.4 Codificació One-Hot (DictVectorizer)

Utilitzem `DictVectorizer` per convertir les variables categòriques en format one-hot. 
**Important:** Fem `fit()` només sobre el conjunt d'entrenament!

In [11]:
# Codificació one-hot amb DictVectorizer (fit només sobre train!)
dv = DictVectorizer(sparse=False)

# Convertir a diccionari i aplicar DictVectorizer
X_train_cat = dv.fit_transform(X_train_raw[cat_cols].to_dict(orient='records'))
X_test_cat = dv.transform(X_test_raw[cat_cols].to_dict(orient='records'))

print("Columnes generades pel DictVectorizer:")
print(dv.get_feature_names_out())
print(f"\nForma X_train_cat: {X_train_cat.shape}")
print(f"Forma X_test_cat: {X_test_cat.shape}")

Columnes generades pel DictVectorizer:
['island=Biscoe' 'island=Dream' 'island=Torgersen' 'sex=Female' 'sex=Male']

Forma X_train_cat: (266, 5)
Forma X_test_cat: (67, 5)


### 3.5 Normalització (StandardScaler)

Apliquem `StandardScaler` a les variables numèriques per estandarditzar-les (mitjana=0, desviació=1).
**Important:** Fem `fit()` només sobre el conjunt d'entrenament!

In [12]:
# Normalització estàndard (fit només sobre train!)
scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train_raw[num_cols])
X_test_num = scaler.transform(X_test_raw[num_cols])

print("Estadístiques del Scaler (calculades sobre train):")
print(f"  Mitjanes: {scaler.mean_.round(2)}")
print(f"  Desviacions: {scaler.scale_.round(2)}")
print(f"\nForma X_train_num: {X_train_num.shape}")
print(f"Forma X_test_num: {X_test_num.shape}")

Estadístiques del Scaler (calculades sobre train):
  Mitjanes: [  43.98   17.23  201.3  4224.44]
  Desviacions: [  5.47   1.97  14.01 808.91]

Forma X_train_num: (266, 4)
Forma X_test_num: (67, 4)


### 3.6 Combinar Features

In [13]:
# Combinar features categòriques i numèriques
X_train = np.hstack([X_train_cat, X_train_num])
X_test = np.hstack([X_test_cat, X_test_num])

print("="*50)
print("RESUM DEL PREPROCESSAMENT")
print("="*50)
print(f"Forma final X_train: {X_train.shape}")
print(f"Forma final X_test: {X_test.shape}")
print(f"Forma y_train: {y_train.shape}")
print(f"Forma y_test: {y_test.shape}")
print(f"\nTotal de features: {X_train.shape[1]}")
print(f"  - Features categòriques (one-hot): {X_train_cat.shape[1]}")
print(f"  - Features numèriques: {X_train_num.shape[1]}")

RESUM DEL PREPROCESSAMENT
Forma final X_train: (266, 9)
Forma final X_test: (67, 9)
Forma y_train: (266,)
Forma y_test: (67,)

Total de features: 9
  - Features categòriques (one-hot): 5
  - Features numèriques: 4


## 4. Entrenament dels Models

Entrenarem 4 models de classificació:
1. **Regressió Logística**
2. **SVM (Support Vector Machine)**
3. **Arbres de Decisió**
4. **KNN (K-Nearest Neighbors)**

In [14]:
# Definir els 4 models
models = {
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42),
    'svm': SVC(kernel='rbf', random_state=42),
    'decision_tree': DecisionTreeClassifier(random_state=42),
    'knn': KNeighborsClassifier(n_neighbors=5)
}

# Entrenar i avaluar cada model
results = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"MODEL: {name.upper()}")
    print('='*60)
    
    # Entrenar
    model.fit(X_train, y_train)
    
    # Predir
    y_pred = model.predict(X_test)
    
    # Avaluar
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"\nAccuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred))


MODEL: LOGISTIC_REGRESSION

Accuracy: 0.9851 (98.51%)

Classification Report:
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        29
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           0.99        67
   macro avg       0.98      0.99      0.98        67
weighted avg       0.99      0.99      0.99        67


MODEL: SVM

Accuracy: 1.0000 (100.00%)

Classification Report:
              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        29
   Chinstrap       1.00      1.00      1.00        14
      Gentoo       1.00      1.00      1.00        24

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


MODEL: DECISION_TREE

Accuracy: 0.9254 (92.54%)

Classification Report:
              precisio

### Resum de Resultats

In [15]:
# Resum comparatiu dels models
print("="*60)
print("RESUM COMPARATIU DELS MODELS")
print("="*60)

results_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': list(results.values())
}).sort_values('Accuracy', ascending=False)

results_df['Accuracy %'] = (results_df['Accuracy'] * 100).round(2)
print(results_df.to_string(index=False))

# Millor model
best_model = results_df.iloc[0]['Model']
best_accuracy = results_df.iloc[0]['Accuracy']
print(f"\n✓ Millor model: {best_model} amb {best_accuracy*100:.2f}% d'accuracy")

RESUM COMPARATIU DELS MODELS
              Model  Accuracy  Accuracy %
                svm  1.000000      100.00
logistic_regression  0.985075       98.51
                knn  0.985075       98.51
      decision_tree  0.925373       92.54

✓ Millor model: svm amb 100.00% d'accuracy


## 5. Serialització dels Models amb Pickle

Guardarem els 4 models entrenats i els preprocessadors (DictVectorizer i StandardScaler) per poder-los utilitzar al servidor Flask.

In [16]:
# Crear directori models si no existeix (ruta relativa des de notebooks/)
MODELS_PATH = '../models'
os.makedirs(MODELS_PATH, exist_ok=True)

# Guardar els 4 models
for name, model in models.items():
    filepath = f'{MODELS_PATH}/{name}.pck'
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)
    print(f"✓ Model guardat: {filepath}")

# Guardar preprocessadors (IMPORTANT per fer prediccions!)
with open(f'{MODELS_PATH}/dict_vectorizer.pck', 'wb') as f:
    pickle.dump(dv, f)
print(f"✓ DictVectorizer guardat: {MODELS_PATH}/dict_vectorizer.pck")

with open(f'{MODELS_PATH}/scaler.pck', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ StandardScaler guardat: {MODELS_PATH}/scaler.pck")

print("\n" + "="*50)
print("SERIALITZACIÓ COMPLETADA")
print("="*50)

✓ Model guardat: ../models/logistic_regression.pck
✓ Model guardat: ../models/svm.pck
✓ Model guardat: ../models/decision_tree.pck
✓ Model guardat: ../models/knn.pck
✓ DictVectorizer guardat: ../models/dict_vectorizer.pck
✓ StandardScaler guardat: ../models/scaler.pck

SERIALITZACIÓ COMPLETADA


In [17]:
# Verificar els fitxers guardats
print("Fitxers al directori models/:")
for f in os.listdir(MODELS_PATH):
    filepath = f'{MODELS_PATH}/{f}'
    size = os.path.getsize(filepath)
    print(f"  {f}: {size} bytes")

Fitxers al directori models/:
  logistic_regression.pck: 966 bytes
  knn.pck: 27249 bytes
  svm.pck: 5586 bytes
  decision_tree.pck: 3217 bytes
  dict_vectorizer.pck: 307 bytes
  scaler.pck: 700 bytes
